## **Decision Trees**

Decision Trees are capable of finding  **complex non-linear relationships** in the data and can be used to perform both **classification** and **regression** tasks.

The **tree** itself is a model in decision trees and we would like to estimate an **optimal tree structure** from the given training data.

* The internal nodes contains **conditions on the features**. Depending on the outcome of the comparision, we take an appropriate path in the tree. The process is repeated **until we reach the leaf node**. 

* In case of **classification**, leaf nodes contains **label** and in case of **regression**, the prediction is obtained by taking **sample mean of labels** of the subset of the training examples present in that leaf node. 



### **Objective**

In this notebook, we will implement decision tree for classification with **ID3 algorithm**.

### **Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import export_text

import warnings  
warnings.filterwarnings('ignore')

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here `eps` is the smallest representable number. At times we get `log(0)` or `0` in the denominator, to avoid that we are going to use this. 

### **Loading dataset**

For this purplose we will use synthetic dataset.

In [3]:
data_dict = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy'], 
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'], 
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'], 
    'Wind': ['False', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'True'], 
    'play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

data = pd.DataFrame(data_dict, columns=data_dict.keys())
data

,Outlook,Temperature,Humidity,Wind,play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rainy,Mild,High,False,Yes
4,Rainy,Cool,Normal,False,Yes
5,Rainy,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
9,Rainy,Mild,Normal,False,Yes


In [4]:
data.shape

(14, 5)

In [5]:
data.keys()

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'play'], dtype='object')

In [6]:
target = data.keys()[-1]
target

'play'

Lets check total number of labels.

In [7]:
data[target].unique()

array(['No', 'Yes'], dtype=object)

There are two labels `yes` and `no`.

In [8]:
print(data[target].unique()[0])
print(data[target].unique()[1])

No
Yes


In [9]:
print(data[target].value_counts()[data[target].unique()[0]])
print(data[target].value_counts()[data[target].unique()[1]])

5
9


9 out of 14 examples have label `yes` and remaining 5 have label `no`.

### **Calculating Entropy of the whole dataset**

Since the formula for information gain requires the entropy of the whole dataset, we compute that now :

In [10]:
def find_entropy_whole(df):
    # last column in dataframe is the target variable
    target = df.keys()[-1]

    # initialization 
    overall_entropy = 0

    # possible values of target 
    values_in_target = df[target].unique()

    for value in values_in_target:
        p = df[target].value_counts()[value] / len(df[target])
        overall_entropy += -p * np.log2(p)
    return overall_entropy

In [11]:
find_entropy_whole(data)

0.9402859586706311

### **Calculating Entropy of an attribute**

In [12]:
def find_entropy_of_attribute(df, attribute):
    # last column in dataframe is the target variable
    target = df.keys()[-1]

    # initialization
    attribute_entropy = 0

    # possible values of target
    values_in_target = df[target].unique()

    # gives possible features in that attribute 
    values_in_attribute = df[attribute].unique()

    for value_in_attribute in values_in_attribute:
        overall_entropy = 0

        for value_in_target in values_in_target:
            num = len(df[attribute][df[attribute] == value_in_attribute][df[target] == value_in_target])

            den = len(df[attribute][df[attribute] ==
                      value_in_attribute])

            p = num / (den + eps)
            overall_entropy += -p * np.log2(p+eps)
        
        p2 = den / len(df)
        attribute_entropy += -p2 * overall_entropy 
    return abs(attribute_entropy)


Let's compute entropy of different attributes now : 

In [14]:
for i in data.keys()[:-1]:
    print(f'Entropy of attribute \'{i}\':',find_entropy_of_attribute(data, i))

Entropy of attribute 'Outlook': 0.6935361388961914
Entropy of attribute 'Temperature': 0.9110633930116756
Entropy of attribute 'Humidity': 0.7884504573082889
Entropy of attribute 'Wind': 0.892158928262361


#### **Finding the best attribute**

In [15]:
def find_best_attribute_to_divide(df):
    # information gain
    ig = []

    # all column names 
    all_attribute_names = df.keys()[:-1]

    for attribute in all_attribute_names:
        # compute information gain for every attribute
        ig.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attribute))

    # get the index of attribute with best information gain 
    index_of_attribute_with_max_ig = np.argmax(ig)

    #print(index_of_attribute_with_max_ig)
    best_attribute = all_attribute_names[index_of_attribute_with_max_ig]
    return best_attribute 

In [16]:
find_best_attribute_to_divide(data)

'Outlook'

### **Building the decison tree**

In [17]:
def buildTree(df, tree=None):
    # last column in dataframe is the target variable
    target = df.keys()[-1]

    # get attribute with best information gain 
    node = find_best_attribute_to_divide(df)

    # get distinct values of that attribute 
    attValue = np.unique(df[node])

    # create an empty dictionary to create tree 
    if tree is None:
        tree = {}
        tree[node] = {}

    # we make loop to construct a tree by calling this function recursively. 
    # In this we check if the subset is pure and stops if it is pure.
    for value in attValue:
        subtable = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtable['play'], return_counts=True)

        if len(counts) == 1: 
            tree[node][value] = clValue[0]
        else:
            # calling the function recursively
            tree[node][value] = buildTree(subtable)

    return tree


In [18]:
buildTree(data)

{'Outlook': {'Overcast': 'Yes',
  'Rainy': {'Wind': {'False': 'Yes', 'True': 'No'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

##### **Notes :**

* ID3 in its pure form perform no backtracking in its search. 

* Once it selects an attribute to test at a particular level in the tree, it never backtracks to reconsider this choice.

* Therefore, it is susceptible to the usual risks of hill-climbing search without backtracking : converging to locally optimal solutions that are not globally optimal.